In [ ]:
!pip install transformers datasets


In [ ]:
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict
from sklearn.model_selection import train_test_split
import torch


In [ ]:
import pandas as pd
from datasets import Dataset
# Load your dataset
df = pd.read_csv('IMDB Dataset.csv')

# Convert sentiment to numerical labels
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# Split the dataset
train_df, test_df = train_test_split(df, test_size=0.25)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples['review'], padding='max_length', truncation=True, max_length=256)  # Reduced max_length


tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/37500 [00:00<?, ? examples/s]

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPT2ForSequenceClassification
import torch



class GPT2ForSequenceClassificationCustom(GPT2ForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        labels=None,
    ):
        # Call the original forward method
        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )

        # Extract the logits
        logits = outputs.logits

        # Compute loss if labels are provided and loss is not already computed
        if labels is not None and 'loss' not in outputs:
            # Define the loss function
            loss_fct = torch.nn.CrossEntropyLoss()
            # Compute loss using logits and labels
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs.loss = loss

        return outputs



# Initialize the custom model instead of the standard GPT2ForSequenceClassification
model = GPT2ForSequenceClassificationCustom.from_pretrained('gpt2', num_labels=2)
model.config.pad_token_id = tokenizer.eos_token_id


Some weights of GPT2ForSequenceClassificationCustom were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install accelerate -U


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['review'], padding='max_length', truncation=True, max_length=512)

# Map tokenization over the whole dataset
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Ensure labels are included
tokenized_train_dataset = tokenized_train_dataset.map(lambda examples: {'labels': examples['sentiment']}, batched=True)
tokenized_test_dataset = tokenized_test_dataset.map(lambda examples: {'labels': examples['sentiment']}, batched=True)


Map:   0%|          | 0/37500 [00:00<?, ? examples/s]

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

Map:   0%|          | 0/37500 [00:00<?, ? examples/s]

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy='steps',
    logging_steps=50,
    save_strategy='no',
    gradient_accumulation_steps=2,
    fp16=True,
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
)

trainer.train()


In [ ]:
trainer.train()


In [ ]:
trainer.evaluate()
